# TODO
- kodowanie cykliczne dla key
- K-means
- metoda mini batch???
- DBSCAN
- GMM


# z GPT

Projekt clusteringu do rekomendacji piosenek na danych ze Spotify jest świetnym pomysłem. Wykorzystanie różnych modeli clusteringowych, takich jak K-means, DBSCAN i GMM, to dobry start. Aby uzupełnić analizę, możesz dodać jeszcze kilka innych metod i kroków:

## Dodatkowe Metody Clusteringu
### Hierarchiczne Clustering (Agglomerative Clustering):

Przydatne dla małych i średnich zbiorów danych.
Tworzy hierarchię klastrów, co pozwala na wizualizację w formie dendrogramu.
### Mean Shift Clustering:

Nie wymaga wstępnego określenia liczby klastrów.
Dobrze radzi sobie z odkrywaniem klastrów o różnej gęstości.
### Spectral Clustering:

Może wychwycić bardziej złożone struktury w danych.
Szczególnie skuteczne, gdy dane mają nieregularne kształty.
## Porównywanie Modeli
Aby porównać modele, możesz użyć następujących metod:

### Metryki Wewnętrzne:

Silhouette Score: Miara zgodności punktów z ich klastrem.
Davies-Bouldin Index: Średnia miara "podobieństwa" każdego klastra z najbardziej podobnym klastrem.
Calinski-Harabasz Index: Stosunek sumy wewnątrzklastrowej a międzyklastrowej.
### Metryki Zewnętrzne (jeśli masz etykiety ground truth):

Adjusted Rand Index (ARI): Miara podobieństwa między dwoma przypisaniami klastrów.
Normalized Mutual Information (NMI): Miara jakości klasteryzacji w odniesieniu do prawdziwych etykiet.
## Wizualizacje
### Redukcja wymiarowości:

PCA (Principal Component Analysis): Do redukcji wymiarowości i wizualizacji danych w 2D/3D.
t-SNE (t-Distributed Stochastic Neighbor Embedding): Świetna metoda do wizualizacji w 2D klastrów, zwłaszcza gdy dane mają wiele wymiarów.
UMAP (Uniform Manifold Approximation and Projection): Alternatywa dla t-SNE, często lepiej zachowująca globalną strukturę danych.
### Wizualizacje przestrzenne:

Wykresy rozrzutu (scatter plots): Wykorzystujące PCA, t-SNE lub UMAP do pokazania klastrów w 2D lub 3D.
Wizualizacje klastrów na mapie cieplnej (heatmap): Do pokazania podobieństw między klastrami.
### Dendrogramy:

Wizualizacja hierarchicznego clusteringu do pokazania relacji między klastrami.
### Macierz pomyłek (Confusion Matrix):

Przydatna, gdy masz etykiety ground truth.